In [1]:
# import libraries
import os
import cv2
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

2022-06-09 18:45:01.862750: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rkfd/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-06-09 18:45:01.862798: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# dataset utils and preprocessing
def image_loader(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, image_size, cv2.INTER_CUBIC)

    return image

In [3]:
def dataset_preprocessing(dataset_path, labels_file_path, image_size, image_paths_pickle):
    with open(labels_file_path, "r") as f:
        classes = f.read().split("\n")[:-1]

    images = []
    labels = []
    image_paths = []

    for image_name in os.listdir(dataset_path):
        try:
            image_path = os.path.join(dataset_path, image_name)
            images.append(image_loader(image_path, image_size))
            image_paths.append(image_path)

            for idx in range(len(classes)):
                if classes[idx] in image_name:
                    labels.append(idx)
        
        except:
            pass

    with open(image_paths_pickle + ".pickle", "wb") as f:
        pickle.dump(image_paths, f)

    assert len(images) == len(labels)
    return np.array(images), np.array(labels)

In [4]:
images, labels = dataset_preprocessing("dataset/train/", "dataset/labels.txt", (32,32), "training_images_pickle")
images.shape

(50000, 32, 32, 3)

In [ ]:
def cosine_distance(training_set_vectors, query_vector, top_n=50):
    distances = []

    for i in range(len(training_set_vectors)):
        distances.append(cosine(training_set_vectors[i]), query_vector[0])
    
    return np.argsort(distances)[:top_n]

In [ ]:
def hamming_distance(training_set_vectors, query_vector, top_n=50):
    distances = []

    for i in range(len(training_set_vectors)):
        distances.append(hamming(training_set_vectors[i]), query_vector[0])
    
    return np.argsort(distances)[:top_n]

In [ ]:
def sparse_accuracy(true_labels, predicted_labels):
    assert len(true_labels) == len(predicted_labels)

    correct = 0

    for i in range(len(true_labels)):
        if np.argmax(predicted_labels[i] == true_labels[i]):
            correct += 1
    
    return correct/len(true_labels)

In [ ]:
def model_inputs(image_size):
    inputs = tf.placeholder(dtype=tf.float32, slope=[None, image_size[0], image_size[1], 3], name='images')
    targets = tf.placeholder(dtype=tf.int32, shape=[None], name='targets')
    dropout_rate = tf.placeholder(dtype=tf.float32, name='dropout_rate')

    return inputs, targets, dropout_rate